<a href="https://colab.research.google.com/github/jeevannani1705/Project-with-ML/blob/main/Random_Forest_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into colaboratory:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 4.2 MB 8.7 MB/s 
     |████████████████████████████████| 1.2 MB 70.2 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
     |████████████████████████████████| 596 kB 79.3 MB/s 
     |████████████████████████████████| 6.6 MB 63.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=0f00c9420347f63bf1210f3986838924b01003bb353038835130d1ea874f1680
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import nltk
# nltk.download('punkt')
import sentence_transformers
from sentence_transformers import SentenceTransformer, models
data_set=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PROJECT WITH ML 2022-JEEVAN/global-headline-only.csv')[['event_type', 'cleaned']]
data_set.head(1)

,event_type,cleaned
0,Explosions/Remote violence,regime forces shelled Arnaba town in Idleb cou...


In [ ]:
print (f"Shape of original data set is {data_set.shape}")

Shape of original data set is (1034527, 2)


In [ ]:
df=data_set.sample(100000).reset_index().drop(['index'], axis = 1)
df.head(1)

,event_type,cleaned
0,Protests,members of the Akhil Bharatiya Hindu Mahasabha...


In [ ]:
#del data_set

In [ ]:
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

In [ ]:
train = model.encode(df.cleaned, device='cuda')

In [ ]:
from sklearn import preprocessing
# features_trainl encode the target variable
encoder = preprocessing.LabelEncoder()
df["event_type_num"] = encoder.fit_transform(df["event_type"])

In [ ]:
df_features = pd.DataFrame()
df_features['cleaned_emb'] = [i for i in train] 
df_features['event_type'] = df["event_type_num"]
df_features.head(3)

,cleaned_emb,event_type
0,"[0.005881261, 0.3087305, -0.065677784, 0.33343...",1
1,"[-0.16774893, -0.014492469, -0.0888344, -0.003...",3
2,"[-0.012007894, -0.22346306, -0.0023450144, -0....",2


In [ ]:
 df_features.to_csv("./features_set.csv")

In [ ]:
from sklearn.decomposition import PCA
import time
pca_model = PCA(n_components=50)
start = time.process_time()
pca_model.fit(train)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))
print(time.process_time() - start)

Sum of variance ratios:  0.5968166115998758
16.842925186000002


In [ ]:
x_comps = pca_model.transform(train)
x_comps.shape

(100000, 50)

**Model Training**

Random Forest

In [ ]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x_comps,df_features['event_type'],test_size=0.2,random_state=42)

In [ ]:
# More algorithms!!!!
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB

rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
print("Score of RFC",rfc.score(x_test,y_test))



Score of RFC 0.8932


In [ ]:
model = GaussianNB()


In [ ]:
RandomForestClassifier.score(x_test,y_test)

TypeError: ignored

In [ ]:
y_pred = RandomForestClassifier.predict(x_test)

TypeError: ignored

In [ ]:
print(y_pred)
print(y_test)

NameError: ignored

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_true= y_test, y_pred= y_pred))

NameError: ignored

K FOLD

In [ ]:
# Making the Confusion Matrix 
from sklearn.metrics import confusion_matrix 
cm = confusion_matrix(y_test, y_pred)

In [ ]:
# Applying k-Fold Cross Validation 
from sklearn.model_selection import cross_val_score 
accuracies = cross_val_score(estimator = svm_classifier, X = x_train, y = y_train, cv = 10) 
accuracies.mean() 
accuracies.std()

In [ ]:
!pip install matplotlib-venn

In [ ]:
!apt-get -qq install -y libfluidsynth1

In [ ]:
# https://pypi.python.org/pypi/libarchive
!apt-get -qq install -y libarchive-dev && pip install -U libarchive
import libarchive

In [ ]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install pydot
import pydot

In [ ]:
!pip install cartopy
import cartopy

In [ ]:
import pandas as pd
import sklearn
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
scores = cross_val_score(
svm_classifier, x_train, y_train, cv=5, scoring='f1_macro')

In [ ]:
from sklearn import model_selection
from sklearn.model_selection import KFold

In [ ]:
results = model_selection.cross_val_score(svm_classifier,x_train,y_train, cv=KFold)

In [ ]:
# More algorithms!!!!
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB,BernoulliNB

rfc = RandomForestClassifier()
rfc.fit(x_train,y_train)
print("Score of RFC",rfc.score(x_test,y_test))

logreg = LogisticRegression()
logreg.fit(x_train,y_train)
print("Score of LogReg",logreg.score(x_test,y_test))

gnb = GaussianNB()
gnb.fit(x_train,y_train)
print("Score of GaussianNB",gnb.score(x_test,y_test))

bnb = BernoulliNB()
bnb.fit(x_train,y_train)
print("Score of BernoulliNB",bnb.score(x_test,y_test))